In [ ]:
import cv2
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from utils.multiscale import YOLOMS
from utils.evaluate import evaluate
import os

class_names = ["BIRD", "DRONE"]

In [ ]:
def plot_detections(img, boxes, labels, scores, title):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(1)
    ax.imshow(img_rgb)
    ax.set_title(title)
    ax.axis('off')

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        width = x2 - x1
        height = y2 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth=2,
                                 edgecolor='lime', facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1 - 5, f'{label} - {score:.2f}', color='yellow', fontsize=8,
                bbox=dict(facecolor='black', alpha=0.5, pad=1, edgecolor='none'))

    return fig

In [ ]:
def read_yolo_annotation(txt_path, img_shape):
    h, w = img_shape[:2]
    boxes = []
    labels = []
    with open(txt_path, 'r') as f:
        for line in f:
            class_id, xc, yc, bw, bh = map(float, line.strip().split())
            x1 = (xc - bw / 2) * w
            y1 = (yc - bh / 2) * h
            x2 = (xc + bw / 2) * w
            y2 = (yc + bh / 2) * h
            boxes.append([x1, y1, x2, y2])
            labels.append(int(class_id))

    return boxes, labels

def ground_truth_detections(img, txt_path):
    gt_boxes, gt_labels = read_yolo_annotation(txt_path, img.shape)
    label_strs = [class_names[l] for l in gt_labels]
    scores = [1.0] * len(gt_boxes)

    plot_detections(img.copy(), gt_boxes, label_strs, scores, "Ground Truth")
    plt.show()

In [ ]:
def yolo_detections(img, model):
    res = model.predict(img,
                        imgsz=[320, 256],
                        device=1,
                        verbose=False)[0]
    boxes_yolo = res.boxes.xyxy.cpu().numpy()
    labels_yolo = res.boxes.cls.cpu().numpy().astype(int)
    scores_yolo = res.boxes.conf.cpu().numpy()
    
    label_strs = [class_names[l] for l in labels_yolo]

    plot_detections(img.copy(), boxes_yolo, label_strs, scores_yolo, "YOLOv8n Original (NMS)")
    plt.show()


In [ ]:
def yolo_multiscale_detections(img, model):
    boxes_yolo_wbf, scores_yolo_wbf, labels_yolo_wbf = model.detect(img, device=1)
    # print(f"YOLOv8n Multiscale: {len(boxes_yolo_wbf)} detections")
    # print(f"Boxes: {boxes_yolo_wbf}")
    # print(f"Scores: {scores_yolo_wbf}")
    # print(f"Labels: {labels_yolo_wbf}")
    label_strs = [class_names[int(l)] for l in labels_yolo_wbf]
    plot_detections(img.copy(), boxes_yolo_wbf, label_strs, scores_yolo_wbf, "YOLOv8n Multiscale")
    plt.show()

In [ ]:
def list_images(txt_file: str):
    paths = []
    with open(txt_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            paths.append(line)
    return paths

In [ ]:
yolo_model = YOLO("/home/cvpr2025/yolo_testing/runs/detect/train/weights/best.pt")
multi_scale_model = YOLOMS(yolo_model,
                           segment_ratio = 0.7,
                           iou_thr = 0.5,
                           conf_thr = 0.5,
                           weight = [1, 2]
                           )

In [ ]:
img_paths = list_images("/home/cvpr2025/yolo_testing/config/Detection/RGB/test.txt")

img_name = "BIRD_02079_103"
img = cv2.imread(f"/home/cvpr2025/yolo_testing/data/vipcup_det/split_A/RGB/images/test/{img_name}.jpg")

In [ ]:
# ground_truth_detections(img, f"/home/cvpr2025/yolo_testing/data/vipcup_det/split_A/RGB/labels/test/{img_name}.txt")
# yolo_detections(img, model=yolo_model)
# yolo_multiscale_detections(img, model=multi_scale_model)

In [ ]:
# res = yolo_model.val(data="/home/cvpr2025/yolo_testing/config/Detection/RGB/vipcup_det_A_RGB.yml", split="test", device=1, verbose=True)

In [ ]:
BASE = '/home/cvpr2025/yolo_testing/data/vipcup_det/split_A/RGB'
IMAGE_FOLDER = os.path.join(BASE, 'images', 'test')
GT_JSON = '/home/cvpr2025/yolo_testing/data/vipcup_det/split_A/RGB/gt_test.json'
CLASS_NAMES = ['BIRD', 'DRONE']

In [ ]:
# print("=== Evaluate YOLOv8")
# res_yolo = evaluate(yolo_model, GT_JSON, IMAGE_FOLDER, device="cpu")

In [ ]:
print("=== Evaluate YOLOv8n Multiscale ===")
res_yolo_multiscale = evaluate(multi_scale_model, GT_JSON, IMAGE_FOLDER, device="cpu")